In [2]:
import numpy as np
from tqdm import tqdm
from tqdm import trange
import jsonlines
from scipy import sparse
from os import listdir
import itertools
import re
from collections import Counter
import json
from scipy.sparse import load_npz

from sklearn.model_selection import KFold
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

In [3]:
ok_set = set()
with open("sources/accepted_categories.txt", mode="r", encoding="utf-8") as inp:
    for line in inp:
        line = line[:-1]
        ok_set.add(line)

ok = sorted(ok_set)

categories_dict = {}
with open("sources/article_cat.json", mode="r") as input:
    categories_dict = json.loads(input.read())

### Тут tf-idf

In [4]:
texts = {item['id']: item['text'] for item in jsonlines.open('sources/normalized_texts.jl', 'r')}

In [5]:
vectorizer = TfidfVectorizer(min_df=5, max_df=1000)
text_tfidf = vectorizer.fit_transform([text for (id, text) in sorted(texts.items())])

In [6]:
ids_tfidf = sorted([id for id in texts.keys()])
svd = TruncatedSVD(n_components=500)
X_tfidf = svd.fit_transform(text_tfidf)

In [10]:
X = X_tfidf
ids = ids_tfidf

kf = KFold(n_splits=3, shuffle=True)


train_index, test_index = 0, 0
for item in kf.split(X):
    train_index, test_index = item[0], item[1]
    break
    
    
print("TRAIN:", train_index, "TEST:", test_index)
print(len(train_index), len(test_index))

TRAIN: [ 3  4  5  6  7  8  9 10 11 13] TEST: [ 0  1  2 12 20]
10 5


In [8]:
Y = sparse.dok_matrix((len(ids), len(ok)), dtype=np.int)
for i in trange(len(ids)):
    for c in categories_dict[ids[i]]:
        Y[i, ok.index(c)] = 1
        
Y = Y.toarray()

100%|██████████████████████████████████████████████████████████████████████████| 96794/96794 [00:30<00:00, 3179.26it/s]


In [12]:
y_predict = []
ids_train = [ids[i] for i in train_index]
X_train = X[train_index]
X_test = X[test_index]

for category in tqdm(ok):
    clf_logreg = LogisticRegression()
    y_train = np.array([1 if category in categories_dict[id] else 0 for id in ids_train])
    if sum(y_train) > 0:
        clf_logreg.fit(X_train, y_train)
        res = clf_logreg.predict(X_test)
        y_predict.append(res)
    else:
        y_predict.append(np.zeros(len(test_index)))

100%|█████████████████████████████████████████████████████████████████████████| 14697/14697 [00:00<00:00, 96433.68it/s]


In [19]:
print(precision_score(np.array(y_predict).transpose(), Y[test_index], average='micro'))
print(recall_score(np.array(y_predict).transpose(), Y[test_index], average='micro'))
print(f1_score(np.array(y_predict).transpose(), Y[test_index], average='micro'))

0.0


C:\Users\Maxim\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


0.0


C:\Users\Maxim\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


0.0
